In [1]:
import sys
sys.path.append("library_app")

In [2]:
from library_app.app import app
app.run(debug=True, use_reloader=False)


 * Serving Flask app 'library_app.app'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [28/Apr/2025 14:16:12] "GET / HTTP/1.1" 302 -
127.0.0.1 - - [28/Apr/2025 14:16:12] "GET /login HTTP/1.1" 200 -


Look at the camera. Press SPACE to capture.
✅ Face captured and saved to captured_face.jpg
✅ Match found: jessica_smith.jpg


127.0.0.1 - - [28/Apr/2025 14:16:41] "GET /face-login HTTP/1.1" 302 -
127.0.0.1 - - [28/Apr/2025 14:16:41] "GET /member/ HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2025 14:16:49] "GET /member/view-books HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2025 14:16:51] "GET /static/qr_codes/3011.png HTTP/1.1" 304 -
127.0.0.1 - - [28/Apr/2025 14:16:51] "GET /static/qr_codes/3012.png HTTP/1.1" 304 -
127.0.0.1 - - [28/Apr/2025 14:16:51] "GET /static/qr_codes/3005.png HTTP/1.1" 304 -
127.0.0.1 - - [28/Apr/2025 14:16:51] "GET /static/qr_codes/3006.png HTTP/1.1" 304 -
127.0.0.1 - - [28/Apr/2025 14:16:51] "GET /static/qr_codes/3007.png HTTP/1.1" 304 -
127.0.0.1 - - [28/Apr/2025 14:16:51] "GET /static/qr_codes/3008.png HTTP/1.1" 304 -
127.0.0.1 - - [28/Apr/2025 14:16:51] "GET /static/qr_codes/3009.png HTTP/1.1" 304 -
127.0.0.1 - - [28/Apr/2025 14:16:51] "GET /static/qr_codes/3001.png HTTP/1.1" 304 -
127.0.0.1 - - [28/Apr/2025 14:16:51] "GET /static/qr_codes/3003.png HTTP/1.1" 304 -
127.0.0.1 - - [28/Apr/2025 14:

In [5]:
from utils.db import get_connection
import oracledb

conn = get_connection()
cursor = conn.cursor()

try:
    # 1. Reset AvailableCopies to TotalCopies
    cursor.execute("UPDATE Books SET AvailableCopies = TotalCopies")
    print("✅ AvailableCopies reset to match TotalCopies.")

    # 2. Clear Loans and Transactions
    cursor.execute("DELETE FROM Loans")
    cursor.execute("DELETE FROM Transactions")
    print("✅ Cleared Loans and Transactions tables.")

    # 3. Drop and recreate sequences
    try:
        cursor.execute("DROP SEQUENCE Loan_SEQ")
        print("✅ Loan_SEQ dropped.")
    except oracledb.DatabaseError as e:
        print("⚠️ Loan_SEQ drop skipped:", e)

    try:
        cursor.execute("DROP SEQUENCE Transaction_SEQ")
        print("✅ Transaction_SEQ dropped.")
    except oracledb.DatabaseError as e:
        print("⚠️ Transaction_SEQ drop skipped:", e)

    # Recreate sequences with correct starting values
    cursor.execute("CREATE SEQUENCE Loan_SEQ START WITH 4001 INCREMENT BY 1")
    cursor.execute("CREATE SEQUENCE Transaction_SEQ START WITH 5001 INCREMENT BY 1")
    print("✅ Sequences recreated starting at 4001 and 5001.")

    conn.commit()
    print("🎉 Reset completed successfully.")

except Exception as e:
    print("❌ Something went wrong:", e)

finally:
    cursor.close()
    conn.close()

✅ AvailableCopies reset to match TotalCopies.
✅ Cleared Loans and Transactions tables.
✅ Loan_SEQ dropped.
✅ Transaction_SEQ dropped.
✅ Sequences recreated starting at 4001 and 5001.
🎉 Reset completed successfully.


In [3]:
from utils.db import get_connection

conn = get_connection()
cursor = conn.cursor()

# Drop the table if it exists
try:
    cursor.execute("DROP TABLE Transactions CASCADE CONSTRAINTS")
    print("✅ Dropped existing Transactions table.")
except Exception as e:
    print("⚠️ Could not drop Transactions table:", e)

# Recreate Transactions with correct allowed TransactionType values
cursor.execute("""
CREATE TABLE Transactions (
    TransactionID INT PRIMARY KEY,
    MemberID INT NOT NULL,
    BookID INT NOT NULL,
    TransactionType VARCHAR2(20) 
        CHECK (TransactionType IN ('Reservation', 'Checkout', 'Return', 'Fine Payment')),
    TransactionDate DATE DEFAULT SYSDATE,
    Amount NUMBER(6,2) DEFAULT 0 CHECK (Amount >= 0),
    Status VARCHAR2(20) 
        CHECK (Status IN ('Pending', 'Completed', 'Cancelled')),
    LibrarianID INT,
    FOREIGN KEY (MemberID) REFERENCES LibraryMembers(MemberID),
    FOREIGN KEY (BookID) REFERENCES Books(BookID),
    FOREIGN KEY (LibrarianID) REFERENCES Librarians(LibrarianID)
)
""")

conn.commit()
cursor.close()
conn.close()
print("✅ Transactions table recreated with all transaction types allowed.")


✅ Dropped existing Transactions table.
✅ Transactions table recreated with all transaction types allowed.


In [5]:
conn = get_connection()
cursor = conn.cursor()

cursor.execute("""
CREATE OR REPLACE TRIGGER trg_calculate_fine
BEFORE UPDATE ON Loans
FOR EACH ROW
BEGIN
    -- Late return fine
    IF :NEW.Status = 'Returned' AND :NEW.ReturnDate > :NEW.DueDate THEN
        :NEW.FineAmount := (:NEW.ReturnDate - :NEW.DueDate) * 1;
    ELSE
        :NEW.FineAmount := 0;
    END IF;

    -- Damage or Lost fee
    IF :NEW.Status = 'Returned' AND :NEW.ReturnCondition = 'Damaged' THEN
        :NEW.DamageFee := 20;
    ELSIF :NEW.Status = 'Returned' AND :NEW.ReturnCondition = 'Lost' THEN
        :NEW.DamageFee := 50;
    ELSE
        :NEW.DamageFee := 0;
    END IF;
END;
""")

conn.commit()
conn.close()

In [3]:
import oracledb

# Connect to the Oracle DB
conn = oracledb.connect(
    user="S25_km242",
    password="uc38wTKk",
    dsn="oracle2.wiu.edu:1521/orclpdb1"  # e.g., "localhost/orclpdb"
)
cursor = conn.cursor()

# 1. View_AdminAllTransactions
cursor.execute("""
CREATE OR REPLACE VIEW View_AdminAllTransactions AS
SELECT 
    T.TransactionID,
    M.FirstName || ' ' || M.LastName AS MemberName,
    B.Title AS BookTitle,
    T.TransactionType,
    TO_CHAR(T.TransactionDate, 'YYYY-MM-DD') AS TransactionDate,
    T.Amount,
    T.Status,
    T.LibrarianID
FROM Transactions T
JOIN Books B ON T.BookID = B.BookID
JOIN LibraryMembers M ON T.MemberID = M.MemberID
ORDER BY T.TransactionDate DESC
""")

# 2. View_AdminAllLoans
cursor.execute("""
CREATE OR REPLACE VIEW View_AdminAllLoans AS
SELECT 
    L.LoanID,
    M.FirstName || ' ' || M.LastName AS MemberName,
    B.Title AS BookTitle,
    TO_CHAR(L.LoanDate, 'YYYY-MM-DD') AS LoanDate,
    TO_CHAR(L.DueDate, 'YYYY-MM-DD') AS DueDate,
    TO_CHAR(L.ReturnDate, 'YYYY-MM-DD') AS ReturnDate,
    L.Status,
    L.FineAmount,
    L.DamageFee,
    L.ReturnCondition
FROM Loans L
JOIN Books B ON L.BookID = B.BookID
JOIN LibraryMembers M ON L.MemberID = M.MemberID
ORDER BY L.LoanDate DESC
""")

# 3. View_AdminAllReservations
cursor.execute("""
CREATE OR REPLACE VIEW View_AdminAllReservations AS
SELECT 
    T.TransactionID,
    M.FirstName || ' ' || M.LastName AS MemberName,
    B.Title AS BookTitle,
    TO_CHAR(T.TransactionDate, 'YYYY-MM-DD') AS ReservationDate,
    T.Status
FROM Transactions T
JOIN Books B ON T.BookID = B.BookID
JOIN LibraryMembers M ON T.MemberID = M.MemberID
WHERE T.TransactionType = 'Reservation'
ORDER BY T.TransactionDate DESC
""")

conn.commit()
cursor.close()
conn.close()

print("✅ All admin views created successfully.")

✅ All admin views created successfully.


In [2]:
import oracledb
from utils.db import get_connection  # assuming you already have this

conn = get_connection()
cursor = conn.cursor()

# List of ALTER TABLE commands
alter_commands = [
    # Books table
    """
    ALTER TABLE Books ADD (IsActive NUMBER(1) DEFAULT 1 CHECK (IsActive IN (0, 1)))
    """,

    # LibraryMembers table
    """
    ALTER TABLE LibraryMembers ADD (IsActive NUMBER(1) DEFAULT 1 CHECK (IsActive IN (0, 1)))
    """,

    # Librarians table
    """
    ALTER TABLE Librarians ADD (IsActive NUMBER(1) DEFAULT 1 CHECK (IsActive IN (0, 1)))
    """,

    # LibraryBranches table
    """
    ALTER TABLE LibraryBranches ADD (IsActive NUMBER(1) DEFAULT 1 CHECK (IsActive IN (0, 1)))
    """
]

# Execute each command
for command in alter_commands:
    try:
        cursor.execute(command)
        print("Executed successfully:", command.strip().split("\n")[0])
    except oracledb.DatabaseError as e:
        print("Error:", e)

conn.commit()
cursor.close()
conn.close()

Executed successfully: ALTER TABLE Books ADD (IsActive NUMBER(1) DEFAULT 1 CHECK (IsActive IN (0, 1)))
Executed successfully: ALTER TABLE LibraryMembers ADD (IsActive NUMBER(1) DEFAULT 1 CHECK (IsActive IN (0, 1)))
Executed successfully: ALTER TABLE Librarians ADD (IsActive NUMBER(1) DEFAULT 1 CHECK (IsActive IN (0, 1)))
Executed successfully: ALTER TABLE LibraryBranches ADD (IsActive NUMBER(1) DEFAULT 1 CHECK (IsActive IN (0, 1)))


In [4]:
from utils.db import get_connection

# Define the sequence creation queries
sequence_queries = [
    "CREATE SEQUENCE seq_books START WITH 3013 INCREMENT BY 1",
    "CREATE SEQUENCE seq_members START WITH 1006 INCREMENT BY 1",
    "CREATE SEQUENCE seq_librarians START WITH 2004 INCREMENT BY 1",
    "CREATE SEQUENCE seq_branches START WITH 4 INCREMENT BY 1"
]

# Execute each query
conn = get_connection()
cursor = conn.cursor()

for query in sequence_queries:
    try:
        cursor.execute(query)
        print(f"✅ Successfully created: {query.split()[1]}")
    except Exception as e:
        print(f"⚠️ Failed to execute '{query}':", e)

conn.commit()
cursor.close()
conn.close()

⚠️ Failed to execute 'CREATE SEQUENCE seq_books START WITH 3013 INCREMENT BY 1': ORA-00955: name is already used by an existing object
Help: https://docs.oracle.com/error-help/db/ora-00955/
⚠️ Failed to execute 'CREATE SEQUENCE seq_members START WITH 1006 INCREMENT BY 1': ORA-00955: name is already used by an existing object
Help: https://docs.oracle.com/error-help/db/ora-00955/
⚠️ Failed to execute 'CREATE SEQUENCE seq_librarians START WITH 2004 INCREMENT BY 1': ORA-00955: name is already used by an existing object
Help: https://docs.oracle.com/error-help/db/ora-00955/
⚠️ Failed to execute 'CREATE SEQUENCE seq_branches START WITH 4 INCREMENT BY 1': ORA-00955: name is already used by an existing object
Help: https://docs.oracle.com/error-help/db/ora-00955/


In [6]:
import oracledb
from utils.db import get_connection

conn = get_connection()
cursor = conn.cursor()

# Step 1: Clear the table
cursor.execute("DELETE FROM AuditLog")
print("🧼 Cleared AuditLog table.")

# Step 2: Drop sequence (ignore error if it doesn't exist)
try:
    cursor.execute("DROP SEQUENCE auditlog_seq")
    print("🔁 Dropped old sequence.")
except oracledb.DatabaseError as e:
    print("⚠️ Sequence might not exist or already dropped.")

# Step 3: Recreate sequence from 1
cursor.execute("""
    CREATE SEQUENCE auditlog_seq
    START WITH 1
    INCREMENT BY 1
    NOCACHE
    NOCYCLE
""")
print("✅ Recreated sequence starting from 1.")

conn.commit()
cursor.close()
conn.close()

🧼 Cleared AuditLog table.
🔁 Dropped old sequence.
✅ Recreated sequence starting from 1.


In [ ]:
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.stats.stattools import jarque_bera

# Ljung-Box test for autocorrelation
ljung_box_results = acorr_ljungbox(results_var.resid, lags=[10], return_df=True)
print(ljung_box_results)

# Jarque-Bera test for normality
jb_statistic, jb_pvalue, _, _ = jarque_bera(results_var.resid)
print("Jarque-Bera Test Statistic:", jb_statistic)
print("Jarque-Bera Test p-value:", jb_pvalue)
